In [9]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
# Load the dataset
uber_data = pd.read_csv('uber.csv')

# Convert pickup_datetime to datetime format
uber_data['pickup_datetime'] = pd.to_datetime(uber_data['pickup_datetime'], errors='coerce')

# Drop rows with missing values
uber_data_cleaned = uber_data.dropna()

# Convert passenger_count to integer
uber_data_cleaned['passenger_count'] = uber_data_cleaned['passenger_count'].astype(int)

# Extract features from the pickup_datetime
uber_data_cleaned['hour'] = uber_data_cleaned['pickup_datetime'].dt.hour
uber_data_cleaned['day'] = uber_data_cleaned['pickup_datetime'].dt.day
uber_data_cleaned['month'] = uber_data_cleaned['pickup_datetime'].dt.month
uber_data_cleaned['year'] = uber_data_cleaned['pickup_datetime'].dt.year
uber_data_cleaned['day_of_week'] = uber_data_cleaned['pickup_datetime'].dt.dayofweek

# Select features and target variable
features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 
            'passenger_count', 'hour', 'day', 'month', 'year', 'day_of_week']
X = uber_data_cleaned[features]
y = uber_data_cleaned['fare_amount']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build a simple TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error: {mae}')

# Save the model
model.save('uber_fare_model')

C:\Users\manoj\AppData\Local\Temp\ipykernel_6732\3995073957.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uber_data_cleaned['passenger_count'] = uber_data_cleaned['passenger_count'].astype(int)
C:\Users\manoj\AppData\Local\Temp\ipykernel_6732\3995073957.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uber_data_cleaned['hour'] = uber_data_cleaned['pickup_datetime'].dt.hour
C:\Users\manoj\AppData\Local\Temp\ipykernel_6732\3995073957.py:15: SettingWithCopyWarning: 
A value is trying to be set on a 

Epoch 1/10
5000/5000 [==============================] - 17s 3ms/step - loss: 140.6800 - mae: 6.0179 - val_loss: 103.4630 - val_mae: 5.9277
Epoch 2/10
5000/5000 [==============================] - 14s 3ms/step - loss: 101.2421 - mae: 5.9643 - val_loss: 102.6761 - val_mae: 5.7953
Epoch 3/10
5000/5000 [==============================] - 14s 3ms/step - loss: 95.3055 - mae: 5.9691 - val_loss: 102.8003 - val_mae: 5.8610
Epoch 4/10
5000/5000 [==============================] - 14s 3ms/step - loss: 96.2983 - mae: 5.9622 - val_loss: 102.3177 - val_mae: 5.9431
Epoch 5/10
5000/5000 [==============================] - 14s 3ms/step - loss: 95.4966 - mae: 5.9701 - val_loss: 102.5056 - val_mae: 6.1086
Epoch 6/10
5000/5000 [==============================] - 14s 3ms/step - loss: 95.4442 - mae: 5.9561 - val_loss: 102.3003 - val_mae: 6.1222
Epoch 7/10
5000/5000 [==============================] - 14s 3ms/step - loss: 95.4553 - mae: 5.9659 - val_loss: 102.1530 - val_mae: 6.0203
Epoch 8/10
5000/5000 [==========

INFO:tensorflow:Assets written to: uber_fare_model\assets


In [14]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_saved_model('uber_fare_model')
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('uber_fare_model.tflite', 'wb') as f:
    f.write(tflite_model)